In [23]:
%pylab inline
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
from sklearn import grid_search
alexnet_train = np.load("CS5785-final-data/alexnet_feat_train.npy")
sift_train = np.load("CS5785-final-data/SIFTBoW_train.npy")
attribute = np.genfromtxt("CS5785-final-data/attributes_train.txt",dtype='str')[:,1]
train = np.genfromtxt("CS5785-final-data/train.txt",dtype='str')
alexnet_test = np.load("CS5785-final-data/alexnet_feat_test.npy")
test_id = np.genfromtxt("CS5785-final-data/test.txt",dtype='str')
train_label = list(train[:,1])

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [46]:
import csv
with open("CS5785-final-data/trainLabel.csv", 'rb') as f:
    train_label = []
    for line in f:
        line = line.strip().split(",")
        train_label.append(line)
#        print [int(a) for a in attr]


In [49]:
print len(train_label)

13000


In [22]:
print len(train[6])

82211


In [27]:
alexTrain = np.load("CS5785-final-data/alexnet_feat_train.npy")
alexunsup = np.load('CS5785-final-data/alexnet_feat_10k.npy') 
alexMerged = np.vstack((alexTrain, alexunsup))


In [29]:
print alexMerged.shape

(13000, 4096)


In [39]:
from sklearn.preprocessing import scale
from sklearn import cross_validation
from sklearn.naive_bayes import BernoulliNB
#from sklearn.preprocessing import normalize
#normal_alexnet = normalize(alexnet_train,norm='l2',axis=1,copy=True)
scale_alexnet = scale(alexMerged, axis=0, with_mean=True, with_std=True, copy=True)
scale_alexnet_test = scale(alexnet_test, axis=0, with_mean=True, with_std=True, copy=True)

In [56]:
train_label = np.array(train_label).reshape((len(train_label),))
print train_label.shape

(13000,)


In [57]:
for a in [0.01,0.1,1,10,100]:
    for b in [0.1,0.3,0.5,0.7,0.9]:
        bnb = BernoulliNB(alpha=a, binarize=b, fit_prior=True, class_prior=None)
#        bnb.fit(scale_alexnet,train_label)
        score = cross_validation.cross_val_score(bnb,scale_alexnet,train_label,cv=5)
        print a,b,score.mean()

0.01 0.1 0.591300048593
0.01 0.3 0.583941850175
0.01 0.5 0.570233331941
0.01 0.7 0.555133320934
0.01 0.9 0.542490070082
0.1 0.1 0.594765255264
0.1 0.3 0.588508448717
0.1 0.5 0.575122007836
0.1 0.7 0.559399202957
0.1 0.9 0.548462502286
1 0.1 0.588166413333
1 0.3 0.579383500803
1 0.5 0.562197775387
1 0.7 0.545087578443
1 0.9 0.524015567971
10 0.1 0.522111699988
10 0.3 0.490322377808
10 0.5 0.471361562279
10 0.7 0.452073200097
10 0.9 0.422865182311
100 0.1 0.481406934797
100 0.3 0.432362836945
100 0.5 0.401041762823
100 0.7 0.393929118844
100 0.9 0.392499528074


In [62]:
from sklearn.naive_bayes import GaussianNB
for a in [0.01,0.1,1,10,100]:
    for b in [0.1,0.3,0.5,0.7,0.9]:
        gnb = GaussianNB()
#        bnb.fit(scale_alexnet,train_label)
        score = cross_validation.cross_val_score(gnb,scale_alexnet,train_label,cv=5)
        print a,b,score.mean()

0.01 0.1 0.576153400441
0.01 0.3 0.576153400441
0.01 0.5 0.576153400441


KeyboardInterrupt: 

In [64]:
gnb = GaussianNB()
gnb.fit(scale_alexnet,train_label)
predicts = gnb.predict(scale_alexnet_test)
print predicts

['lawn' 'rubble' 'forest_road' 'harbor' 'tower' 'forest_road'
 'television_studio' 'shoe_shop' 'junk_pile' 'aquatic_theater' 'cliff'
 'tea_garden' 'building_facade' 'roof' 'windmill' 'swamp' 'forest_road'
 'rubble' 'raceway' 'living_room' 'glacier' 'closet' 'gorge' 'drugstore'
 'fishpond' 'science_museum' 'banquet_hall' 'piano_store' 'forest_road'
 'submarine_interior' 'bus_interior' 'science_museum' 'sawmill'
 'forest_road' 'dining_car' 'manhole' 'recreation_room' 'orchard'
 'voting_booth' 'bus_interior' 'bamboo_forest' 'dorm_room' 'town_house'
 'amusement_park' 'yard' 'building_facade' 'yard' 'airplane_cabin'
 'hallway' 'coffee_shop' 'shoe_shop' 'voting_booth' 'physics_laboratory'
 'fort' 'rock_arch' 'rainforest' 'airplane_cabin' 'windmill' 'sawmill'
 'tea_garden' 'pet_shop' 'raceway' 'manhole' 'rainforest' 'hallway' 'attic'
 'canteen' 'forest_road' 'music_studio' 'dentists_office' 'bayou' 'canteen'
 'campsite' 'ocean' 'newsroom' 'art_school' 'voting_booth' 'cemetery'
 'bookstore' 'a

In [60]:
import csv
with open("semi_supervised_result.txt",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["ID","Category"])
    for i in range(len(predicts)):
        writer.writerow([test_id[i],predicts[i]])

In [66]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.005, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100,verbose=0)
score = cross_validation.cross_val_score(logreg,scale_alexnet,train_label,cv=5)
print score.mean()

0.693601619374


In [67]:
logreg.fit(scale_alexnet,train_label)
predicts = logreg.predict(scale_alexnet_test)

In [68]:
print predicts

['soccer_field' 'forest_road' 'forest_road' 'harbor' 'tower' 'forest_road'
 'forest_road' 'shoe_shop' 'forest_road' 'aquatic_theater' 'forest_road'
 'ranch' 'fire_station' 'forest_road' 'windmill' 'swamp' 'forest_road'
 'forest_road' 'raceway' 'reception' 'glacier' 'elevator_lobby' 'gorge'
 'butchers_shop' 'forest_road' 'forest_road' 'airplane_cabin' 'piano_store'
 'forest_road' 'food_court' 'bus_interior' 'science_museum'
 'airplane_cabin' 'forest_road' 'dining_car' 'forest_road' 'office'
 'orchard' 'closet' 'airplane_cabin' 'bamboo_forest' 'dorm_room'
 'town_house' 'airplane_cabin' 'yard' 'town_house' 'yard' 'airplane_cabin'
 'hallway' 'parlor' 'armory' 'airplane_cabin' 'physics_laboratory' 'fort'
 'forest_road' 'rainforest' 'airplane_cabin' 'windmill' 'sawmill'
 'tea_garden' 'videostore' 'raceway' 'manhole' 'rainforest' 'hallway'
 'forest_road' 'dorm_room' 'forest_road' 'chemistry_lab' 'boat_deck'
 'gas_station' 'airplane_cabin' 'airplane_cabin' 'ocean' 'art_school'
 'art_school' 'f

In [69]:
import csv
with open("semi_supervised_logreg_result.txt",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["ID","Category"])
    for i in range(len(predicts)):
        writer.writerow([test_id[i],predicts[i]])